In [11]:
using JuMP
using GLPK
using MAT
using CSV
using DataFrames

# Define the model
model = Model(GLPK.Optimizer)


# Load data from MATLAB file
mat_file = matopen("c:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/Senario generation/Falling prob.mat")
load_shedding_prob = read(mat_file)
close(mat_file)


# Load the Excel file
df = CSV.read("C:/Users/HP/OneDrive - Imperial College London/Planning for Extreme Weather event/optimisation/IEEE 33.csv",DataFrame)

# Convert DataFrame columns to arrays for easier use
branch_no = df[!, "Branch No"]
sending_end = df[!, "Sending End"]
receiving_end = df[!, "Receiving End"]
R = df[!, "R (ohms)"]
X = df[!, "X (ohms)"]
PL = df[!, "PL (kW)"]
QL = df[!, "QL (kVAR)"]

# Define sets
ΩN = 1:33  # Example set of buses
ΩB = 1:33  # Example set of lines
#ΩDG = 1:5  # Example set of buses with DGs
#ΩSW = 1:10  # Example set of lines with switches
T = 1:2:12  # Example set of time periods
S = 1:1  # Example set of scenarios

# Define parameters (replace these with actual values)
#cc = Dict((i, j) => 10.0 for i in ΩB, j in ΩB) #annual capital cost for adding an automatic switch
ch = Dict((i, j) => 2000.0 for i in ΩB, j in ΩB) #hardening lines
#cg = Dict(i => 100.0 for i in ΩDG) #deploying a back-up DG
cL = Dict(i => 14.0 for i in ΩN) #penalty cost for load-shedding
cR0 = 1000.0 #base repair cost
ωH = 2.0 #average occurance per year
pr = 1
#pr = Dict(s => 0.1 for s in S)  # Example scenario probabilities
#PL = Dict((i, t, s) => rand(100:200) for i in ΩN, t in T, s in S)  # Example loads
#QL = Dict((i, t, s) => rand(50:100) for i in ΩN, t in T, s in S)  # Example reactive loads

# Define first-stage variables
#@variable(model, xg[i in ΩDG], Bin) #new back-up DG
@variable(model, xh[i in ΩB, j in ΩB], Bin) #hardening
#@variable(model, xc[i in ΩB, j in ΩB], Bin) #line switch
#@variable(model, xc1[i in ΩB, j in ΩB], Bin) #new line switch

# Define second-stage variables
@variable(model, γ[i in ΩN, t in T, s in S] >= 0)  # Load shedding percentage
@variable(model, u[i in ΩB, j in ΩB, t in T, s in S], Bin)  # Line damage status

# Define second-stage cost components
@expression(model, repair_cost, sum(cR0 * u[i, j, t, s] for i in ΩB, j in ΩB, t in T, s in S))
@expression(model, load_shedding_cost, sum(cL[i] * γ[i, t, s] * PL[i, t, s] for i in ΩN, t in T, s in S))

# Objective function
@objective(model, Min, sum(ch[i, j] * xh[i, j] for i in ΩB, j in ΩB) +
                        #sum(cg[i] * xg[i] for i in ΩDG) +
                        #sum(cc[i, j] * xc1[i, j] for i in ΩB, j in ΩB) +
                        ωH * sum(pr[s] * (repair_cost + load_shedding_cost) for s in S))

# Constraints
@constraint(model, sum(xg[i] for i in ΩDG) <= length(ΩDG))  # Example constraint for DGs
@constraint(model, [i in ΩB, j in ΩB], xc[i, j] == xc1[i, j])  # Example constraint for switches

# Add other constraints for power flow, voltage limits, and so on

# Solve the model
optimize!(model)

# Print the results
println("Objective value: ", objective_value(model))
for i in ΩDG
    println("xg[$i] = ", value(xg[i]))
end


BoundsError: BoundsError: attempt to access 32-element Vector{Int64} at index [1, 3, 1]